# Анализ вакансий с сайта LinkedIn. Подготовка к визуализации

## Описание данных

### Описание проекта

Для анализа представлены данные с информацией о рынке европейских вакансий для дата аналитика / BI аналитика.  
Данные представлены виде файла .сsv c вакансиями, спарсенными с LinkedIn 07/09/2022 за неделю.  

### Цели и задачи

В рамках проекта необходимо провести анализ вакансий, для чего выполнить задачи:  
1. распарсить csv файл и выделить характерные признаки для каждой вакансии в понятный датафрейм (название вакансии, местоположение, данные о работодателе, тип занятости и т.п.),
2. провести предварительную обработку данных,
3. провести анализ данных с визуализацией и построение дашборда.

В рамках настоящего проекта будут выполнены задачи №№ 1-2. Задача визуализации будет выполнена в Tableau.

## Импорт библиотек и загрузка данных

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_colwidth', 70)

In [4]:
df = pd.read_csv('masterskaya_yandex_2022_09_07.csv', index_col=0)
df['html'][0]

'\n  <div>\n    <div class="\n        jobs-details__main-content jobs-details__main-content--single-pane full-width\n        ">\n        \n<!---->\n      \n<!---->\n      <div>\n        <div class="jobs-unified-top-card t-14">\n  <div class="relative">\n\n    <div class="jobs-unified-top-card__content--two-pane">\n<!---->\n        <a href="/jobs/view/3258155313/?alternateChannel=search&amp;refId=xQWMq5sHxPa4BOVKDp0TKg%3D%3D&amp;trackingId=grGqx%2FHW0EZ%2BorBvEr%2Ft%2Bg%3D%3D&amp;trk=d_flagship3_search_srp_jobs" id="ember464" class="ember-view">\n          <h2 class="t-24 t-bold jobs-unified-top-card__job-title">Stage - Assistant Ingénieur Qualité - Beyrand (H/F)</h2>\n        </a>\n\n      <div class="jobs-unified-top-card__primary-description">\n          <span class="jobs-unified-top-card__subtitle-primary-grouping t-black">\n            <span class="jobs-unified-top-card__company-name">\n                  <a href="/company/hermes-group/life/" id="ember465" class="ember-view t-black 

## Распарсивание данных

### Навазние вакансии

Выведем из html названия вакансий

In [5]:
df['title'] = df['html'].apply(lambda x: bs(x).find('h2').text.strip())

In [6]:
df.head()

,html,title
0,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Stage - Assistant Ingénieur Qualité - Beyrand (H/F)
1,"\n <div>\n <div class=""\n jobs-details__main-content jo...","développeur matlab/simulink, secteur automobile f/h"
2,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst
3,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst - Belgium
4,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst


### Местоположение вакансии

Далее распарсим местоположение вакансии

In [7]:
def get_place(cell):
    try:
        return bs(cell).find('span', class_='jobs-unified-top-card__bullet').text.strip()
    except:
        np.nan

df['place'] = df['html'].apply(get_place)

In [8]:
df.head()

,html,title,place
0,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Stage - Assistant Ingénieur Qualité - Beyrand (H/F),"Limoges, Nouvelle-Aquitaine, France"
1,"\n <div>\n <div class=""\n jobs-details__main-content jo...","développeur matlab/simulink, secteur automobile f/h","Toulouse, Occitanie, France"
2,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst,"Skara, Vastra Gotaland County, Sweden"
3,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst - Belgium,"West Flanders, Flemish Region, Belgium"
4,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,"Mecklenburg-West Pomerania, Germany"


Из полученного местоположения выделим в отдельные столбцы название города и страну

In [9]:
# город

def get_city(cell):
    if len(cell.split(',')) > 1:
        return cell.split(',')[0].strip()
    elif "Metropolitan" in cell or "Greater" in cell:
        return cell.replace('Greater', '').replace('Metropolitan', '').replace('Area', '').replace('Region','').strip()
    else:
        return np.nan

df['city'] = df['place'].apply(get_city)

In [10]:
# страна

def get_country(cell):
    if len(cell.split(',')) > 1:
        return cell.split(',')[-1].strip()
    elif "Metropolitan" in cell or "Greater" in cell or "Region" in cell:
        return np.nan
    else:
        return cell
    
df['country'] = df['place'].apply(get_country)

In [11]:
df.head()

,html,title,place,city,country
0,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Stage - Assistant Ingénieur Qualité - Beyrand (H/F),"Limoges, Nouvelle-Aquitaine, France",Limoges,France
1,"\n <div>\n <div class=""\n jobs-details__main-content jo...","développeur matlab/simulink, secteur automobile f/h","Toulouse, Occitanie, France",Toulouse,France
2,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst,"Skara, Vastra Gotaland County, Sweden",Skara,Sweden
3,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst - Belgium,"West Flanders, Flemish Region, Belgium",West Flanders,Belgium
4,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,"Mecklenburg-West Pomerania, Germany",Mecklenburg-West Pomerania,Germany


### Тип занятости

Далее выделим тип занятости

In [12]:
def work_type(cell):
    try:
        return bs(cell).find('span', class_='jobs-unified-top-card__workplace-type').text.strip()
    except:
        np.nan

df['work_type'] = df['html'].apply(work_type)


### Данные о компании-работодателе

Найдем и сохраним в отдельные столбцы данные о компании - её название, размер (количество сотрудников) и сферу деятельности

In [13]:
# название компании

def company_name(cell):
    try:
        return bs(cell).find('span', class_='jobs-unified-top-card__company-name').text.strip()
    except:
        np.nan

df['company_name'] = df['html'].apply(company_name)

In [14]:
# размер компании

def company_size(cell):
    try:
        return bs(cell).find('div', class_='mt5 mb2').find_all('li', class_='jobs-unified-top-card__job-insight'
                                               )[1].text.split(' · '
                                                              )[0].replace('\n', '').replace(' employees', '')
    except:
        np.nan

df['company_size'] = df['html'].apply(company_size)

In [15]:
# сфера деятельности компании

def company_area(cell):
    try:
        return bs(cell).find('div', class_='mt5 mb2').find_all('li', class_='jobs-unified-top-card__job-insight'
                                               )[1].text.split(' · '
                                                              )[1].replace('\n', '')
    except:
        np.nan

df['company_area'] = df['html'].apply(company_area)

In [16]:
df.head()

,html,title,place,city,country,work_type,company_name,company_size,company_area
0,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Stage - Assistant Ingénieur Qualité - Beyrand (H/F),"Limoges, Nouvelle-Aquitaine, France",Limoges,France,On-site,Hermès,"10,001+",Retail Luxury Goods and Jewelry
1,"\n <div>\n <div class=""\n jobs-details__main-content jo...","développeur matlab/simulink, secteur automobile f/h","Toulouse, Occitanie, France",Toulouse,France,On-site,AUSY,"5,001-10,000",IT Services and IT Consulting
2,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst,"Skara, Vastra Gotaland County, Sweden",Skara,Sweden,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting
3,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst - Belgium,"West Flanders, Flemish Region, Belgium",West Flanders,Belgium,Remote,TELUS International,"10,001+",IT Services and IT Consulting
4,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,"Mecklenburg-West Pomerania, Germany",Mecklenburg-West Pomerania,Germany,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting


### Hard-skills

Далее выведем в отдельный столбец описание вакансии, чтобы из него выделить hard-skills

In [17]:
# распарсиваем описание вакансии

df['description'] = df['html'].apply(lambda x: bs(x).find('div', {'id':'job-details'}).text.strip())

In [18]:
# создаем список hard-skills

skills = (['datahub', 'api', 'github', 'google analytics', 'adobe analytics', 'ibm coremetrics', 'omniture',
            'gitlab', 'erwin', 'hadoop', 'spark', 'hive', 'databricks', 'aws', 'gcp', 'azure','excel',
            'redshift', 'bigquery', 'snowflake',  'hana', 'grafana', 'kantar', 'spss', 
           'asana', 'basecamp', 'jira', 'dbeaver','trello', 'miro', 'salesforce', 
           'rapidminer', 'thoughtspot',  'power point',  'docker', 'jenkins','integrate.io', 'talend', 'apache nifi',
          'aws glue','pentaho','google data flow', 'azure data factory','xplenty','skyvia','iri voracity','xtract.io',
          'dataddo', 'ssis', 'hevo data','informatica','oracle data integrator','k2view','cdata sync','querysurge', 
             'rivery', 'dbconvert', 'alooma', 'stitch', 'fivetran', 'matillion','streamsets','blendo',
             'iri voracity','logstash', 'etleap', 'singer', 'apache camel','actian', 'airflow', 'luidgi', 'datastage',
           'python', 'vba', 'scala', ' r ', 'java script', 'julia', 'sql', 'matlab', 'java', 'html', 'c++', 'sas',
           'data studio', 'tableau', 'looker', 'powerbi', 'cognos', 'microstrategy', 'spotfire',
             'sap business objects','microsoft sql server', 'oracle business intelligence', 'yellowfin',
             'webfocus','sas visual analytics', 'targit', 'izenda',  'sisense', 'statsbot', 'panorama', 'inetsoft',
             'birst', 'domo', 'metabase', 'redash', 'power bi', 'alteryx', 'dataiku', 'qlik sense', 'qlikview', 
         'hypotheses', 'ab testing', 'a/b testing', 'ab tests', 'a/b tests', 'statistics', 'statistical analysis'])

In [19]:
# с помощью функции проверяем наличие hard-skills из списка в описании вакансии 
# и сохраняем найденные hard-skills в отдельный столбец

def hard_skills(cell):
    list_skills = []
    for skill in skills:
        if skill in cell.lower().replace('powerbi', 'power bi'):
            list_skills.append(skill)
    return list_skills

df['hard_skills'] = df['description'].apply(hard_skills)

In [20]:
df.head(3)

,html,title,place,city,country,work_type,company_name,company_size,company_area,description,hard_skills
0,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Stage - Assistant Ingénieur Qualité - Beyrand (H/F),"Limoges, Nouvelle-Aquitaine, France",Limoges,France,On-site,Hermès,"10,001+",Retail Luxury Goods and Jewelry,"LA SOCIETE : \nCréée en 1926, la société Beyrand, filiale du Group...","[api, excel]"
1,"\n <div>\n <div class=""\n jobs-details__main-content jo...","développeur matlab/simulink, secteur automobile f/h","Toulouse, Occitanie, France",Toulouse,France,On-site,AUSY,"5,001-10,000",IT Services and IT Consulting,"Dans le cadre de la croissance de nos activités automobile, nous r...",[matlab]
2,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst,"Skara, Vastra Gotaland County, Sweden",Skara,Sweden,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[]


### Иные сведения о вакансии

Также выделим из html данные о дате публикации вакансии и количества откликнувшихся кандидатов

In [21]:
# дата публикации

def publish_date(cell):
    try:
        return bs(cell).find('span', class_='jobs-unified-top-card__posted-date').text.strip()
    except:
        np.nan

df['publish_date'] = df['html'].apply(publish_date)

In [22]:
# количество кандидатов

def get_candidats(cell):
    try:
        return bs(cell).find('span', class_='jobs-unified-top-card__applicant-count'
                            ).text.strip().replace('applicants', '').replace('applicant', '')
    except:
        pd.NA
        
df['candidats'] = df['html'].apply(get_candidats)

## Предобработка данных

### Фильтр по вакансиям

Отфильтруем датасет по релевантным вакансиям

In [23]:
# создадим список релевантных вакансий и проверим совпадение вакансий в столбце title с релевантными вакансиями из списка

relevant_vac = ['Data Analyst', 'Data analyst', 'data analyst', ' Data Analyst' 
                'Product Analyst', 'Product analyst', 'product analyst', 
                'Business Intelligence', 'Business intelligence', 'business intelligence']

vac = []
for i in df['title']:
    for k in relevant_vac:
        if k in i:
            vac.append(i)

In [24]:
# сделаем срез по релевантным вакансиям и сохраним в отдельный датасет

df_rel = df.loc[df['title'].isin(vac)].reset_index(drop=True)

In [25]:
df_rel.shape

(262, 13)

Итого после фильтрации релевантных вакансий у нас осталось 262 строки из 772.

### Заполнение пропусков

Проверим полученный датасет на наличие пропусков

In [26]:
df_rel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   html          262 non-null    object
 1   title         262 non-null    object
 2   place         262 non-null    object
 3   city          245 non-null    object
 4   country       250 non-null    object
 5   work_type     213 non-null    object
 6   company_name  262 non-null    object
 7   company_size  261 non-null    object
 8   company_area  243 non-null    object
 9   description   262 non-null    object
 10  hard_skills   262 non-null    object
 11  publish_date  262 non-null    object
 12  candidats     234 non-null    object
dtypes: object(13)
memory usage: 26.7+ KB


In [27]:
df_rel[df_rel['city'].isna() == True]

,html,title,place,city,country,work_type,company_name,company_size,company_area,description,hard_skills,publish_date,candidats
9,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst (Tableau),Gibraltar,NaN,Gibraltar,None,Guardian Jobs,51-200,Staffing and Recruiting,"GURU CAREERS\n\nData Analysts, who have experience of Tableau & SQ...","[api, excel, python, sql, tableau]",1 day ago,30
12,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,Poland,NaN,Poland,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],8 hours ago,9
31,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst - Belgium,Belgium,NaN,Belgium,Remote,TELUS International,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],6 days ago,11
70,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,Poland,NaN,Poland,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],5 days ago,9
82,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst,Finland,NaN,Finland,Remote,TELUS International,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],6 days ago,15
83,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,Germany,NaN,Germany,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],5 days ago,22
93,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Work From Home | Data Analyst,Poland,NaN,Poland,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],2 days ago,42
110,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,Germany,NaN,Germany,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],6 days ago,28
126,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst - Part time,Poland,NaN,Poland,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],6 days ago,13
158,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst (Tijdelijk - 3 Maanden),Netherlands,NaN,Netherlands,Remote,Independent Recruiters,11-50,Staffing and Recruiting,We are proud to be an equal employer. We are convinced our differe...,[excel],2 days ago,19


Пропуски в столбце city заполнить нельзя из-за отсутствия информации в исходных данных, поэтому оставляем NaN.

In [28]:
df_rel[df_rel['country'].isna() == True]

,html,title,place,city,country,work_type,company_name,company_size,company_area,description,hard_skills,publish_date,candidats
5,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,Przemyśl Metropolitan Area,Przemyśl,NaN,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],8 hours ago,None
13,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,Grudziadz Metropolitan Area,Grudziadz,NaN,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],8 hours ago,3
16,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst | Flexible Work,Greater Norrköping Metropolitan Area,Norrköping,NaN,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],1 day ago,23
17,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,Radom Metropolitan Area,Radom,NaN,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],8 hours ago,7
19,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,Zamosc Metropolitan Area,Zamosc,NaN,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],8 hours ago,5
20,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst | REMOTE OPPORTUNITY,Greater Edinburgh Area,Edinburgh,NaN,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],7 hours ago,30
40,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Remote| Data Analyst,Lodz Metropolitan Area,Lodz,NaN,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],2 days ago,7
60,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Work From Home | Data Analyst,Lodz Metropolitan Area,Lodz,NaN,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],2 days ago,12
96,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Engineer for Business Intelligence/ Data Analytics team,Bucharest Metropolitan Area,Bucharest,NaN,Hybrid,LSEG (London Stock Exchange Group),"10,001+",Financial Services,Main Responsibilities\nWork with stakeholders throughout the organ...,"[aws, excel, redshift, python, sql, tableau]",7 hours ago,10
99,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Junior Data Analyst,Greater Milan Metropolitan Area,Milan,NaN,Hybrid,Pardgroup,"501-1,000",Advertising Services,"Pardgroup SpA, Multinazionale B2B operante nel settore dei Servizi...","[excel, sql, tableau]",4 hours ago,167


Пропуски в странах заполним вручную на основе информации о городах:

In [29]:
df_rel.loc[(df_rel['country'].isna() == True) & (df_rel['city'].isin(
        ['Przemyśl', 'Grudziadz', 'Radom', 'Zamosc', 'Lodz'])), 'country'] = 'Poland'
        
df_rel.loc[(df_rel['country'].isna() == True) & (df_rel['city'].isin(
        ['Norrköping'])), 'country'] = 'Sweden'
        
df_rel.loc[(df_rel['country'].isna() == True) & (df_rel['city'].isin(
        ['Edinburgh'])), 'country'] = 'United Kingdom'
        
df_rel.loc[(df_rel['country'].isna() == True) & (df_rel['city'].isin(
        ['Bucharest'])), 'country'] = 'Romania'

df_rel.loc[(df_rel['country'].isna() == True) & (df_rel['city'].isin(
        ['Milan'])), 'country'] = 'Italy'
        
df_rel.loc[(df_rel['country'].isna() == True) & (df_rel['city'].isin(
        ['Zurich'])), 'country'] = 'Switzerland'
        
df_rel.loc[(df_rel['country'].isna() == True) & (df_rel['city'].isin(
        ['Barcelona'])), 'country'] = 'Spain'

In [30]:
df_rel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   html          262 non-null    object
 1   title         262 non-null    object
 2   place         262 non-null    object
 3   city          245 non-null    object
 4   country       262 non-null    object
 5   work_type     213 non-null    object
 6   company_name  262 non-null    object
 7   company_size  261 non-null    object
 8   company_area  243 non-null    object
 9   description   262 non-null    object
 10  hard_skills   262 non-null    object
 11  publish_date  262 non-null    object
 12  candidats     234 non-null    object
dtypes: object(13)
memory usage: 26.7+ KB


Пропусков в данных о типе занятости немного, поэтому оставим соответствующие поля пустыми.

В столбце company_size некоторые данные заполнены некорректно:

In [31]:
df_rel.company_size.value_counts()

10,001+                                                                                             108
1,001-5,000                                                                                          34
51-200                                                                                               25
201-500                                                                                              22
501-1,000                                                                                            19
5,001-10,000                                                                                         16
1-10                                                                                                 14
11-50                                                                                                12
See recent hiring trends for Software * IT. Unlock more Premium insights for free                     2
See recent hiring trends for www.TeamQuest.pl. Unlock more Premi

В связи с отсутствием иной информации заменим некорректно заполненные 11 значений в столбце company_size на 'no information' - тем самым добавятся пропуски к датафрейму, но данные будут корректными:

In [32]:
df_rel.loc[df_rel['company_size'].isin(['10,001+', '1,001-5,000', '51-200', '201-500', '501-1,000', 
                                        '5,001-10,000', '1-10', '11-50']) == False, 'company_size'] = 'no information'

In [33]:
df_rel.company_size.value_counts()

10,001+           108
1,001-5,000        34
51-200             25
201-500            22
501-1,000          19
5,001-10,000       16
1-10               14
11-50              12
no information     12
Name: company_size, dtype: int64

Пропуски в столбце company_area, company_size и candidats иным способом заполнить не представляется возможным, поэтому оставим их пустыми, тем более что количество пропусков не очень значительное.

### Расчёт даты публикации вакансии

Даты публикации нам даны не в виде дат, а в виде промежутков типа '6 дней назад'. Для последующего анализа необходимо расчитать конретные даты публикаций. Считать будем от даты парсинга - '2022-09-07 17:00:00'.

Для начала обозначим переменной дату парсинга, от которой будем отсчитывать даты публикации вакансий

In [34]:
parsing_date = pd.to_datetime('2022-09-07 17:00:00')

In [35]:
df_rel.publish_date.value_counts()

1 day ago       60
6 days ago      59
5 days ago      46
2 days ago      29
8 hours ago     13
4 days ago      11
23 hours ago     7
2 hours ago      7
3 days ago       6
7 hours ago      5
9 hours ago      4
21 hours ago     4
3 hours ago      2
14 hours ago     1
19 hours ago     1
4 hours ago      1
6 hours ago      1
18 hours ago     1
16 hours ago     1
11 hours ago     1
1 week ago       1
10 hours ago     1
Name: publish_date, dtype: int64

Как видим из имеющихся промежутков у нас есть дни, часы и недели. С помощью функции проверим вхождение соответствующих слов (день, час, неделя) дате публикации каждой из вакансий и отнимем от даты парсинга соответствующее значение, после чего сохраним полученную дату в отдельный столбец date.

In [36]:
def date(cell):
    if 'hours ago' in cell:
        cell = (parsing_date - timedelta(hours=int(cell.strip()[0]))).strftime('%Y-%m-%d')
        return cell
    
    elif 'days ago' in cell or 'day ago' in cell:
        cell = (parsing_date - timedelta(days=int(cell.strip()[0]))).strftime('%Y-%m-%d')
        return cell
    
    elif 'week ago' in cell:
        cell = (parsing_date - timedelta(weeks=int(cell.strip()[0]))).strftime('%Y-%m-%d')
        return cell
    
    else:
        return np.nan

In [37]:
df_rel['date'] = df_rel['publish_date'].apply(date)

In [38]:
df_rel.head()

,html,title,place,city,country,work_type,company_name,company_size,company_area,description,hard_skills,publish_date,candidats,date
0,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst,"Skara, Vastra Gotaland County, Sweden",Skara,Sweden,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],6 days ago,12,2022-09-01
1,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst - Belgium,"West Flanders, Flemish Region, Belgium",West Flanders,Belgium,Remote,TELUS International,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],6 days ago,11,2022-09-01
2,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,"Mecklenburg-West Pomerania, Germany",Mecklenburg-West Pomerania,Germany,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],8 hours ago,2,2022-09-07
3,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,"Hamburg, Germany",Hamburg,Germany,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],8 hours ago,4,2022-09-07
4,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Alternant/ Alternante data analyst M/F,"Rousset, Provence-Alpes-Côte d'Azur, France",Rousset,France,On-site,STMicroelectronics,"10,001+",Semiconductor Manufacturing,"POURQUOI NOUS REJOINDRE\n\nAu sein de ST, nous sommes 48000 hommes...","[excel, python, sql, java, html, spotfire, power bi]",5 days ago,25,2022-09-02


### Проверка на дубликаты

In [39]:
print('Явных дубликатов:', df_rel.duplicated(subset=['html']).sum())

Явных дубликатов: 0


Теперь проверим дубликаты в сочетании 'название вакансии - компания - город - описание':

In [40]:
df_rel[df_rel.duplicated(subset=['title', 'company_name', 'city', 'description'])]

,html,title,place,city,country,work_type,company_name,company_size,company_area,description,hard_skills,publish_date,candidats,date
61,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst - Boursorama,"Boulogne-Billancourt, Île-de-France, France",Boulogne-Billancourt,France,Remote,Boursorama,"501-1,000",Financial Services,"Avec son double positionnement unique de banque-média, Boursorama ...","[excel, python, sas, tableau]",6 days ago,62,2022-09-01
66,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst - Boursorama,"Boulogne-Billancourt, Île-de-France, France",Boulogne-Billancourt,France,Remote,Boursorama,"501-1,000",Financial Services,"Avec son double positionnement unique de banque-média, Boursorama ...","[excel, python, sas, tableau]",6 days ago,63,2022-09-01
68,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst - Boursorama,"Boulogne-Billancourt, Île-de-France, France",Boulogne-Billancourt,France,Remote,Boursorama,"501-1,000",Financial Services,"Avec son double positionnement unique de banque-média, Boursorama ...","[excel, python, sas, tableau]",6 days ago,38,2022-09-01


Удалим дубликаты

In [41]:
df_rel = df_rel.drop_duplicates(subset=['title', 'company_name', 'city', 'description'], keep='last').reset_index(drop=True)
df_rel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   html          259 non-null    object
 1   title         259 non-null    object
 2   place         259 non-null    object
 3   city          242 non-null    object
 4   country       259 non-null    object
 5   work_type     210 non-null    object
 6   company_name  259 non-null    object
 7   company_size  259 non-null    object
 8   company_area  240 non-null    object
 9   description   259 non-null    object
 10  hard_skills   259 non-null    object
 11  publish_date  259 non-null    object
 12  candidats     231 non-null    object
 13  date          259 non-null    object
dtypes: object(14)
memory usage: 28.5+ KB


### Замена типов данных

In [42]:
df_rel.dtypes

html            object
title           object
place           object
city            object
country         object
work_type       object
company_name    object
company_size    object
company_area    object
description     object
hard_skills     object
publish_date    object
candidats       object
date            object
dtype: object

Подавляющее число данных у нас относятся к типу object - из них только данные в столбце candidats можем привести к числовому типу, а в столбце date - к datetime:

In [43]:
df_rel['candidats'] = df_rel['candidats'].astype('float').astype('Int64')
df_rel['date'] = pd.to_datetime(df_rel['date'])

Столбец с hard-skills разбиваем по каждому отдельному hard-skill для корректного построения дашборда.

In [44]:
df_rel = df_rel.reset_index()

In [45]:
df_rel

,index,html,title,place,city,country,work_type,company_name,company_size,company_area,description,hard_skills,publish_date,candidats,date
0,0,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst,"Skara, Vastra Gotaland County, Sweden",Skara,Sweden,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],6 days ago,12,2022-09-01
1,1,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Online Data Analyst - Belgium,"West Flanders, Flemish Region, Belgium",West Flanders,Belgium,Remote,TELUS International,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],6 days ago,11,2022-09-01
2,2,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,"Mecklenburg-West Pomerania, Germany",Mecklenburg-West Pomerania,Germany,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],8 hours ago,2,2022-09-07
3,3,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,"Hamburg, Germany",Hamburg,Germany,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,TELUS International AI-Data Solutions partners with a diverse and ...,[],8 hours ago,4,2022-09-07
4,4,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Alternant/ Alternante data analyst M/F,"Rousset, Provence-Alpes-Côte d'Azur, France",Rousset,France,On-site,STMicroelectronics,"10,001+",Semiconductor Manufacturing,"POURQUOI NOUS REJOINDRE\n\nAu sein de ST, nous sommes 48000 hommes...","[excel, python, sql, java, html, spotfire, power bi]",5 days ago,25,2022-09-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,254,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Data Analyst,"Dublin, County Dublin, Ireland",Dublin,Ireland,On-site,permanent tsb,"1,001-5,000",Banking,"Permanent TSB is one of Ireland's leading retail and SME banks, wi...","[excel, sql]",6 days ago,148,2022-09-01
255,255,"\n <div>\n <div class=""\n jobs-details__main-content jo...",BI Data Analyst Intern (f/m/x),"Berlin, Berlin, Germany",Berlin,Germany,On-site,Sharpist,201-500,Professional Training and Coaching,Job Description\n\nWe are a fast-growing start-up on a journey to ...,"[python, scala, sql, data studio, tableau, looker, power bi]",1 day ago,55,2022-09-06
256,256,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Language Data Analyst,"Cork, County Cork, Ireland",Cork,Ireland,On-site,TELUS International,"10,001+",IT Services and IT Consulting,Role Summary\n\nCome and join our international and multicultural ...,"[excel, scala]",6 days ago,18,2022-09-01
257,257,"\n <div>\n <div class=""\n jobs-details__main-content jo...",Assistenz (w/m/d) des CEO West - Schwerpunkt Data Analyst / Contro...,"Düsseldorf, North Rhine-Westphalia, Germany",Düsseldorf,Germany,On-site,Rhenus Logistics,"10,001+","Transportation, Logistics, Supply Chain and Storage",Gibt es eine Logistik für Karrieren? Bei uns schon. Als weltweit o...,[ssis],2 hours ago,<NA>,2022-09-07


In [51]:
df_fin = df_rel.explode('hard_skills')

In [52]:
df_fin.dtypes

index                    int64
html                    object
title                   object
place                   object
city                    object
country                 object
work_type               object
company_name            object
company_size            object
company_area            object
description             object
hard_skills             object
publish_date            object
candidats                Int64
date            datetime64[ns]
dtype: object

Оставим в датафрейме только необходимые столбцы

In [53]:
df_fin = df_fin[['index', 'title', 'city', 'country', 'work_type', 'company_name', 'company_size', 
                 'company_area', 'hard_skills', 'date', 'candidats']]

In [54]:
df_fin

,index,title,city,country,work_type,company_name,company_size,company_area,hard_skills,date,candidats
0,0,Online Data Analyst,Skara,Sweden,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,NaN,2022-09-01,12
1,1,Online Data Analyst - Belgium,West Flanders,Belgium,Remote,TELUS International,"10,001+",IT Services and IT Consulting,NaN,2022-09-01,11
2,2,Data Analyst,Mecklenburg-West Pomerania,Germany,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,NaN,2022-09-07,2
3,3,Data Analyst,Hamburg,Germany,Remote,TELUS International AI Data Solutions,"10,001+",IT Services and IT Consulting,NaN,2022-09-07,4
4,4,Alternant/ Alternante data analyst M/F,Rousset,France,On-site,STMicroelectronics,"10,001+",Semiconductor Manufacturing,excel,2022-09-02,25
...,...,...,...,...,...,...,...,...,...,...,...
257,257,Assistenz (w/m/d) des CEO West - Schwerpunkt Data Analyst / Contro...,Düsseldorf,Germany,On-site,Rhenus Logistics,"10,001+","Transportation, Logistics, Supply Chain and Storage",ssis,2022-09-07,<NA>
258,258,Corporate Business Intelligence Engineer,Paris,France,On-site,Preligens,51-200,Defense and Space Manufacturing,api,2022-09-01,2
258,258,Corporate Business Intelligence Engineer,Paris,France,On-site,Preligens,51-200,Defense and Space Manufacturing,aws,2022-09-01,2
258,258,Corporate Business Intelligence Engineer,Paris,France,On-site,Preligens,51-200,Defense and Space Manufacturing,excel,2022-09-01,2


In [55]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 802 entries, 0 to 258
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   index         802 non-null    int64         
 1   title         802 non-null    object        
 2   city          771 non-null    object        
 3   country       802 non-null    object        
 4   work_type     588 non-null    object        
 5   company_name  802 non-null    object        
 6   company_size  802 non-null    object        
 7   company_area  729 non-null    object        
 8   hard_skills   728 non-null    object        
 9   date          802 non-null    datetime64[ns]
 10  candidats     701 non-null    Int64         
dtypes: Int64(1), datetime64[ns](1), int64(1), object(8)
memory usage: 76.0+ KB


Итак, после предобработки у нас получился датасет, состоящий из 802 строк и 10 столбцов с релевантными вакансиями, без дубликатов, с понятными датами публикации и корректными типами данных.

## Экспорт в csv

Подготовленный датасет сохраним в файл csv

In [56]:
df_fin.to_csv('linkedin_data.csv', index=False)